In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-10 16:47:48,421 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 16:47:48,422 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BDA8" # rid


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-10 16:47:49,370 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 16:47:49,371 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-10 16:47:50,893 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post18+git.d89701e2.dirty


In [6]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-10 16:47:51,117 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json] transfer successful. 0.68 KB transferred. Elapsed time: 0:00:00.000057.
2024-06-10 16:47:51,118 - INFO - Verifying SHA256 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json]
2024-06-10 16:47:51,133 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_data_analysis',
    'rid': '2-BDAC'}],
  'Execution_Asset_Type': [{'name': 'Multimodal_Analysis', 'rid': '2-BDAE'}]},
 'execution_rid': '2-C60R',
 'workflow_rid': '2-BDAG',
 'bag_paths': [PosixPath('/data/2-7P5P_a14456c09cb78569ae70c30ee54767a9f0b3a4aa0ae9f80871dd9f8ff3f6f3dc/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json')}

In [7]:
# Compute condition labels
icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])

condition_labels = EA.compute_condition_label(icd10_asso, icd10)

,RID,Field_Size,Strategy,False_Neg,False_Pos,Fixation_Loss,Fovea,Layout,MD,PSD,...,Visual_Acuity,GHT,PSD.1,Abs_Perc,Abs_Val,Pat_Perc,Pat_Val,Raw_Val,Report,Image_Side_Vocab
0,2-B93Y,24-2,SITA Standard,5%,7%,5/15,34,v3,-1.93,NaN,...,20/30,Outside Normal Limits,5.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX0,2SK2
1,2-B940,24-2,SITA Fast,Off,0%,NaN,Off,v3,-8.16,NaN,...,NaN,Outside Normal Limits,7.85,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX8,2SK2
2,2-B942,24-2,SITA Standard,0%,0%,0/15,32,v3,-2.87,NaN,...,NaN,Outside Normal Limits,5.12,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXG,2SK2
3,2-B944,24-2,SITA Standard,0%,1%,0/15,38,v3,-0.95,1.74,...,20/30,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXM,2SK2
4,2-B946,24-2,SITA Fast,0%,0%,0/11,37,v3,-1.81,NaN,...,20/20,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXR,2SK2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,2-BD9Y,24-2,SITA Fast,Off,3%,NaN,29,v3,-6.74,NaN,...,20/30,Outside Normal Limits,865.00,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2E6,2SK0
2145,2-BDA0,24-2,SITA Fast,0%,17%XX,1/10,40,v3,1.61,1.25,...,20/25,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EA,2SK0
2146,2-BDA2,24-2,SITA Standard,0%,0%,NaN,36,v3,-0.75,NaN,...,NaN,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EM,2SK0
2147,2-BDA4,24-2,SITA Fast,Off,0%,NaN,34,v3,-3.77,NaN,...,20/30,Outside Normal Limits,2.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EW,2SK0


In [25]:
# Insert new condition labels
EA.insert_condition_label(condition_labele)


,Clinical_Records,Condition_Label
4532,2-7XT4,POAG
0,2-7XT6,POAG
4534,2-7XT8,POAG
2,2-7XTA,POAG
4536,2-7XTC,POAG
...,...,...
4523,2-868W,GS
9063,2-868Y,GS
4526,2-8690,GS
9065,2-8692,GS


In [ ]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)